# Thermal loads

....

In this example, a household whose its only load is the shower.

In [ ]:
# importing functions
from ramp import User,calc_peak_time_range,yearly_pattern
from ramp import load_data
import pandas as pd

### Creating a user category and appliances

In [ ]:
household = User()

when the power is variant during the day, the "power" parameter needs to be passed as a pd.DataFrame or np.array with a daily profile (365 rows of data). For this exercise, data can be loaded from the default examples in ramp:

In [ ]:
shower_power = load_data(example="shower")

In [ ]:
# shower power distribution
shower_power.plot()

In [ ]:
shower = household.Appliance(
    name = "Shower",
    number = 1,
    power  = shower_power,
    num_windows = 2,
    func_time = 15,
    time_fraction_random_variability = 0.1,
    func_cycle = 3,
    thermal_P_var = 1,
)

shower.windows(
    window_1 = [390,540],
    window_2 = [1080,1200],
    random_var_w = 0.2
)


### Generating profiles

In [ ]:
peak_time_range = calc_peak_time_range(
    user_list = [household]
)
year_behaviour = yearly_pattern()

In [ ]:
# generating the yearly profiles for different thermal_P_var

In [ ]:
def thermal_P_var_sensitivity(values):
    
    # buidling a pd.DataFrame for saving sensitivity results
    results = pd.DataFrame(    
        index = pd.date_range(start = "2020-01-01",periods = 1440*365,freq="T"),
        columns = [f"P_var = {value}" for value in values]
    )
    
    for value in values:
        
        # changing the thermal_P_var
        household.thermal_P_var = value
        
        # creating a list to sotre profiles for all the years of the year
        profiles = []
        for day in range(365):

            profile = household.generate_single_load_profile(
                    prof_i = day, 
                    peak_time_range = peak_time_range,
                    Year_behaviour = year_behaviour
                )

            profiles.extend(profile)
            
        # assiging the yearly profile for a given sensitivity case
        results[f"P_var = {value}"] = profiles
        
    return results

In [ ]:
sensitivity_results = thermal_P_var_sensitivity([0,0.25,0.5,0.75,1])

In [ ]:
sensitivity_results

In [ ]:
average_daily_profiles = sensitivity_results.resample("1d").mean()

In [ ]:
average_daily_profiles.plot()